In [1]:
# The code was removed by DSX for sharing.

In [2]:
temp_df = pd.read_csv(project.get_file('hn_stories.csv'))
temp_df.shape

(6229, 11)

In [3]:
temp_df.shape

(6229, 11)

In [4]:
clean_df = temp_df.dropna(axis=0, how='all', subset=['text'])
clean_df['text_length'] = clean_df.text.str.len()
clean_df = clean_df[clean_df.text_length > 200]

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [5]:
# https://www.machinelearningplus.com/nlp/topic-modeling-python-sklearn-examples/
# Author: Olivier Grisel <olivier.grisel@ensta.org>
#         Lars Buitinck
#         Chyi-Kwei Yau <chyikwei.yau@gmail.com>
# License: BSD 3 clause

from __future__ import print_function
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint
import re
import numpy as np

from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn import metrics

from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans, MiniBatchKMeans

%matplotlib inline
# # Plotting tools
# import matplotlib.pyplot as plt
# %matplotlib inline

In [6]:
# Convert to list
data = clean_df.text.tolist()
data = [re.sub(r'[^\w\s]','',s) for s in data] #punctuation
data = [re.sub('\s+', ' ', s) for s in data] #new lines
data = [re.sub("\'", "", s) for s in data] # single quotes
clean_df['text'] = data

In [7]:
n_samples = 2000
n_features = 100000
n_components = 8
n_top_words = 20

In [8]:
from sklearn.pipeline import Pipeline


tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=n_features,
                                   stop_words='english')
svd = TruncatedSVD(n_components)
normalizer = Normalizer(copy=False)

km = KMeans(n_clusters=n_components, init='k-means++', max_iter=100, n_init=1,
                verbose=False)

ward = AgglomerativeClustering(n_clusters=8, linkage='ward')

k_pipe = make_pipeline(tfidf_vectorizer, svd, normalizer,km)
agg_pipe = make_pipeline(tfidf_vectorizer, svd, normalizer,ward)

In [13]:
k_model = k_pipe.fit(clean_df.text)
k_clusts  = k_model.predict(clean_df.text)
clean_df['clust_label'] = clusts

In [14]:
agg_model = agg_pipe.fit(clean_df.text)
agg_clusts  = agg_model.fit_predict(clean_df.text)
clean_df['clust_label_agg'] = clusts

In [15]:
clean_df['clust_label'].value_counts()

0    1611
2    1515
7     650
6     510
4     439
3     402
5     256
1     135
Name: clust_label, dtype: int64

In [16]:
k_model.named_steps['kmeans'].cluster_centers_

array([[  6.05923300e-01,  -1.51241354e-01,  -1.08419520e-01,
          3.66263559e-01,   7.39461672e-02,  -9.89564001e-02,
          2.24678291e-01,   6.78694635e-02],
       [  7.88204320e-01,  -7.53496667e-02,   4.00300442e-01,
         -1.08051264e-01,  -2.11425777e-01,   3.60239305e-02,
          1.59974950e-01,   3.95247965e-03],
       [  8.73804657e-01,  -2.07436633e-01,   4.49768840e-02,
         -2.42215694e-01,   1.34958329e-01,  -1.16450071e-02,
         -5.67063004e-02,  -3.77243384e-03],
       [  4.69157407e-01,   8.42180353e-01,  -4.09749925e-02,
          7.78416802e-03,  -2.18505399e-02,  -6.24424923e-03,
          4.32149430e-03,   1.04013383e-04],
       [  7.72737154e-01,  -2.74848146e-01,  -3.41394671e-01,
         -2.28432271e-01,   4.27334258e-03,  -5.52751426e-02,
          2.08575164e-01,   4.80623414e-03],
       [  7.03263885e-01,  -2.39408394e-01,  -4.12915783e-01,
          1.18534616e-01,  -1.01135701e-01,   1.42191534e-01,
         -3.12706510e-01,   1.1

In [18]:
clusts  = k_model.predict(clean_df.text)
np.unique(clusts,return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7], dtype=int32),
 array([ 584,  544,  881,  274, 1196,  746,  632,  661]))

##  Score all the clustered stories and save to COS bucket

In [19]:
clean_df.head()

,by,descendants,id,kids,pain_perdu,score,text,time,title,type,url,text_length,clust_label,clust_label_agg
0,pain_perdu,77.0,16936533,"[16937090, 16936857, 16936689, 16936691, 16937...",NaN,141,Copyright 2000 2016 Fox Television Stations L...,1524781744,DNA linked to Golden State Killer arrest came ...,story,http://www.ktvu.com/news/dna-linked-to-golden-...,6154,0,0
1,rezashirazian,57.0,16935270,"[16936257, 16935515, 16936338, 16937273, 16935...",NaN,234,var postLoadFunctions var foresee_enabled 1 v...,1524773832,Square to buy Weebly for $365M,story,https://www.cnbc.com/2018/04/26/square-to-buy-...,28245,1,1
2,arctux,109.0,16933736,"[16935012, 16935247, 16935094, 16934190, 16937...",NaN,402,by Matthew Hodgson Apr 26 2018 General In the...,1524762980,Matrix and Riot Confirmed as the Basis for Fra...,story,https://matrix.org/blog/2018/04/26/matrix-and-...,4477,2,2
3,ingve,102.0,16933265,"[16933598, 16933544, 16934620, 16933861, 16934...",NaN,462,Uncovering the best of the web while pointing ...,1524759583,Magnasanti: Large and Terrifying SimCity (2010),story,https://rumorsontheinternets.org/2010/10/14/ma...,9866,0,0
4,velmu,20.0,16934656,"[16935702, 16936638, 16937333, 16935952, 16937...",NaN,92,Cyber security has evolved into a key issue f...,1524769885,"SEC Penalizes Yahoo $35M for Massive, Undisclo...",story,https://www.jdsupra.com/legalnews/sec-penalize...,10789,6,6


In [122]:
clean_df.to_json('clustered_stories.json',orient='records')
cos.upload_file(Filename='clustered_stories.json' ,Bucket=credentials['BUCKET'],Key='clustered_stories.json' )

## Deploy model with Watson Machine Learning

In [20]:
# The code was removed by DSX for sharing.

In [21]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient
from watson_machine_learning_client.metanames import MetaNames
import json

2018-05-10 14:20:37,040 - watson_machine_learning_client.metanames - WARNING - 'AUTHOR_EMAIL' meta prop is deprecated. It will be ignored.
2018-05-10 14:21:26,838 - watson_machine_learning_client.wml_client_error - WARNING - Loading model with artifact_uid: 'a012bfe2-ec80-4097-9c75-ece24c1ab42f' failed.
Reason: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'X-Global-Transaction-Id': 'ffbbzc8fe4dblfb8225p2j4f73rae2398fr2', 'Transfer-Encoding': 'chunked', 'Server': 'nginx/1.13.6', 'X-Xss-Protection': '1', 'Date': 'Thu, 10 May 2018 14:21:26 GMT', 'X-Wml-User-Client': 'PythonClient', 'X-Frame-Options': 'DENY', 'Cache-Control': 'private, no-cache, no-store, must-revalidate', 'Pragma': 'no-cache', 'X-Backside-Transport': 'FAIL FAIL', 'X-Content-Type-Options': 'nosniff', 'Connection': 'Keep-Alive', 'Content-Type': 'application/json'})
HTTP response body: {"trace":"ffbbzc8fe4dblfb8225p2j4f73rae2398fr2","errors":[{"code":"not_found","message":"Requested object could not be foun

In [22]:
client = WatsonMachineLearningAPIClient(wml_credentials)

# instance_details = client.service_instance.get_details()
# print(json.dumps(instance_details, indent=2))

In [23]:
model_props = {client.repository.ModelMetaNames.AUTHOR_NAME: "Greg F.", 
               client.repository.ModelMetaNames.AUTHOR_EMAIL: "gfilla@us.ibm.com",
               client.repository.ModelMetaNames.NAME: "HN Clustering",
               MetaNames.LABEL_FIELD: "binned"}

In [24]:
l =  clean_df.text.values.tolist()

In [26]:
published_model = client.repository.store_model(model=k_model, meta_props=model_props, \
                                                training_data=l, training_target=clean_df.score)

In [27]:
published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)

In [28]:
print(json.dumps(model_details, indent=2))

{
  "metadata": {
    "guid": "48a7da24-8bfc-4b93-82a7-b2e7de8d6d6b",
    "url": "https://ibm-watson-ml.mybluemix.net/v3/wml_instances/bedc993c-0caa-4c43-bd7e-cc3986e45fea/published_models/48a7da24-8bfc-4b93-82a7-b2e7de8d6d6b",
    "modified_at": "2018-05-10T14:20:37.349Z",
    "created_at": "2018-05-10T14:20:37.299Z"
  },
  "entity": {
    "input_data_schema": {
      "features": {
        "fields": [
          {
            "type": "str",
            "name": "f1"
          }
        ],
        "type": "list"
      },
      "labels": {
        "fields": [
          {
            "type": "int64",
            "name": "score"
          }
        ],
        "type": "Series"
      }
    },
    "deployments": {
      "url": "https://ibm-watson-ml.mybluemix.net/v3/wml_instances/bedc993c-0caa-4c43-bd7e-cc3986e45fea/published_models/48a7da24-8bfc-4b93-82a7-b2e7de8d6d6b/deployments",
      "count": 0
    },
    "latest_version": {
      "url": "https://ibm-watson-ml.mybluemix.net/v3/ml_assets/m

In [36]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient
from watson_machine_learning_client.metanames import MetaNames
import json

client = WatsonMachineLearningAPIClient(wml_credentials)
loaded_model = client.repository.load(model_details['metadata']['guid'] )


In [37]:
loaded_model.predict([clean_df.text[0]])

array([4], dtype=int32)

In [38]:
models_details = client.repository.list_models()


loaded_model = client.repository.load(published_model_uid)


------------------------------------  -------------  ------------------------  -----------------
GUID                                  NAME           CREATED                   FRAMEWORK
0a15f965-38c5-4fb4-82b8-f3b87d969019  xj_model       2018-04-09T18:18:23.472Z  spss-modeler-18.1
28f106e5-6650-41a0-837b-55068a1c9d6c  HN Clustering  2018-04-30T22:41:13.971Z  scikit-learn-0.19
48a7da24-8bfc-4b93-82a7-b2e7de8d6d6b  HN Clustering  2018-05-10T14:20:37.299Z  scikit-learn-0.19
------------------------------------  -------------  ------------------------  -----------------


## Using deployed model


In [14]:
end_pt = 'https://ibm-watson-ml.mybluemix.net/v3/wml_instances/bedc993c-0caa-4c43-bd7e-cc3986e45fea/published_models/28f106e5-6650-41a0-837b-55068a1c9d6c/deployments/718598cf-eff3-446d-ae03-a666b7e70c6f/online'

In [18]:
# The code was removed by DSX for sharing.

In [ ]:
payload_scoring = {"fields": ["f1"], "values": [[clean_df.text[0]]]}

response_scoring = requests.post('https://ibm-watson-ml.mybluemix.net/v3/wml_instances/bedc993c-0caa-4c43-bd7e-cc3986e45fea/published_models/28f106e5-6650-41a0-837b-55068a1c9d6c/deployments/718598cf-eff3-446d-ae03-a666b7e70c6f/online', json=payload_scoring, headers=header)
